In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

In [3]:
import bs4
bs4.__version__

'4.11.1'

In [4]:
pd.__version__

'1.5.3'

In [ ]:
# !pip install bs4

In [11]:
def replace_all(text, to_replace):
    for i in to_replace:
        text = text.replace(i, ' ')
    return text

# Парсим песни

In [4]:
url = 'https://ru.investing.com/equities/russia'
songs_list = requests.get(url)
songs_soup = BeautifulSoup(songs_list.content, "html.parser") 
songs_soup

<!DOCTYPE html>

<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]-->
<!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]-->
<!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function

In [60]:
urls = ['https://rustih.ru/viktor-coj/', 'https://rustih.ru/viktor-coj/page/2/']

songs_lists = []

for url in urls:
    songs_list = requests.get(url)
    # songs_list.status_code

    songs_soup = BeautifulSoup(songs_list.content, "html.parser")    
    for a in songs_soup.find_all('a'):
        if str(a).find('viktor-coj') != -1 and str(a).find('page') == -1 and str(a).find('Page') == -1:
            print(a)
            song_url = str(a['href'])
            songs_lists.append(song_url)
            
# songs_lists = list(set(songs_lists))
print(len(songs_lists))

<a href="https://rustih.ru/viktor-coj-vremya-est-a-deneg-net/">Виктор Цой — Время есть, а денег нет</a>
<a href="https://rustih.ru/viktor-coj-prosto-xochesh-ty-znat/">Виктор Цой — Просто хочешь ты знать</a>
<a href="https://rustih.ru/viktor-coj-solnechnye-dni/">Виктор Цой — Солнечные дни</a>
<a href="https://rustih.ru/viktor-coj-alyuminievye-ogurcy/">Виктор Цой — Алюминиевые огурцы</a>
<a href="https://rustih.ru/viktor-coj-bezdelnik/">Виктор Цой — Бездельник</a>
<a href="https://rustih.ru/viktor-coj-bezdelnik-2/">Виктор Цой — Бездельник-2</a>
<a href="https://rustih.ru/viktor-coj-vosmiklassnica/">Виктор Цой — Восьмиклассница</a>
<a href="https://rustih.ru/viktor-coj-moi-druzya/">Виктор Цой — Мои друзья</a>
<a href="https://rustih.ru/viktor-coj-sitar-igral/">Виктор Цой — Ситар играл</a>
<a href="https://rustih.ru/viktor-coj-derevo/">Виктор Цой — Дерево</a>
<a href="https://rustih.ru/viktor-coj-kogda-to-ty-byl-bitnikom/">Виктор Цой — Когда-то ты был битником</a>
<a href="https://rustih.r

In [75]:
parsed_songs = {}



for song_name in songs_lists:
    song_url = requests.get(song_name)
    song_soup = BeautifulSoup(song_url.content, "html.parser")
    song_text = str(str(song_soup.find_all('p')[:len(song_soup.find_all('p'))-2]))
    song_text = replace_all(song_text, ['<p>', '<br/>', '</p>', '\n', '[', ']'])
    song_text = song_text.rstrip().lstrip()
    song_text = " ".join(song_text.split())
    parsed_songs[song_name] = song_text
    
    time.sleep(0.5)
parsed_songs

{'https://rustih.ru/viktor-coj-vremya-est-a-deneg-net/': 'Дождь идет с утра, Будет, был и есть. И карман мой пуст, На часах шесть. Папирос нет, И огня нет, И в окне знакомом Не горит свет. , Время есть, а денег нет, И в гости некуда пойти. , И куда-то все Подевались вдруг. Я попал в какой-то Не такой круг. Я хочу пить, Я хочу есть, Я хочу просто Где-нибудь сесть. , Время есть, а денег нет, И в гости некуда пойти.',
 'https://rustih.ru/viktor-coj-prosto-xochesh-ty-znat/': 'Идешь по улице один, Идешь к кому-то из друзей. Заходишь в гости без причин И просишь свежих новостей. Просто хочешь ты знать, Где и что происходит. , Звонишь по телефону всем, Кого-то — нет, а кто-то — здесь. Для разговоров много тем, Для разговоров время есть. Просто хочешь ты знать, Где и что происходит. , Узнал, что где-то пьют вино, А где-то музыка слышна. Тебя зовут туда, где пьют, И ты берешь еще вина. Просто хочешь ты знать, Где и что происходит. , Там кто-то спор ведет крутой, А кто-то просто спит давно. И с 

# Работа с БД

In [7]:
import sqlite3

In [96]:
con = sqlite3.connect("songs.db") #создали локальную БД

cur = con.cursor() 
cur.execute("CREATE TABLE text_songs(author, name, text, url)")
# res = cur.execute("Select name FROM sqlite_master")

for k, v in parsed_songs.items():
    sql_insert = """
    INSERT INTO text_songs (author, name, text, url)
    VALUES ('{}', '{}', '{}', '{}');
    """.format('Виктор Цой',k[k.find('viktor-coj-')+11:len(k)-1], v, k)
    print(sql_insert)
    
    res = cur.execute(sql_insert)
    con.commit()
# con.close()


    INSERT INTO text_songs (author, name, text, url)
    VALUES ('Виктор Цой', 'vremya-est-a-deneg-net', 'Дождь идет с утра, Будет, был и есть. И карман мой пуст, На часах шесть. Папирос нет, И огня нет, И в окне знакомом Не горит свет. , Время есть, а денег нет, И в гости некуда пойти. , И куда-то все Подевались вдруг. Я попал в какой-то Не такой круг. Я хочу пить, Я хочу есть, Я хочу просто Где-нибудь сесть. , Время есть, а денег нет, И в гости некуда пойти.', 'https://rustih.ru/viktor-coj-vremya-est-a-deneg-net/');
    

    INSERT INTO text_songs (author, name, text, url)
    VALUES ('Виктор Цой', 'prosto-xochesh-ty-znat', 'Идешь по улице один, Идешь к кому-то из друзей. Заходишь в гости без причин И просишь свежих новостей. Просто хочешь ты знать, Где и что происходит. , Звонишь по телефону всем, Кого-то — нет, а кто-то — здесь. Для разговоров много тем, Для разговоров время есть. Просто хочешь ты знать, Где и что происходит. , Узнал, что где-то пьют вино, А где-то музыка слышна.


    INSERT INTO text_songs (author, name, text, url)
    VALUES ('Виктор Цой', 'mama-anarxiya', 'Солдат шел по улице домой И увидел этих ребят. Кто ваша мама, ребята? — Спросил у ребят солдат. , Мама — Анархия, Папа — стакан портвейна. , Все они в кожаных куртках, Все небольшого роста, Хотел солдат пройти мимо, Но это было не просто. , Мама — Анархия, Папа — стакан портвейна. , Довольно веселую шутку Сыграли с солдатом ребята: Раскрасили красным и синим, Заставляли ругаться матом. , Мама — Анархия, Папа — стакан портвейна.', 'https://rustih.ru/viktor-coj-mama-anarxiya/');
    

    INSERT INTO text_songs (author, name, text, url)
    VALUES ('Виктор Цой', 'zhizn-v-steklax', 'Темные улицы тянут меня к себе, Я люблю этот город, как женщину «Х». На улицах люди, и каждый идет один, Я закрываю дверь, я иду вниз. , Я знаю, что здесь пройдет моя жизнь, Жизнь в стеклах витрин, Я растворяюсь в стеклах витрин, Жизнь в стеклах витрин. , И вот я иду, и рядом со мной идут, Я смотрю на них, мне каж


    INSERT INTO text_songs (author, name, text, url)
    VALUES ('Виктор Цой', 'nam-s-toboj', 'Здесь не понятно, где лицо, а где рыло, И не понятно, где пряник, где плеть. Здесь в сено не втыкаются вилы, А рыба проходит сквозь сеть. И не ясно, где море, где суша, Где золото, а где медь. Что построить, и что разрушить, И кому, и зачем здесь петь? , Нам с тобой: голубых небес навес. Нам с тобой: станет лес глухой стеной. Нам с тобой: из оплеванных колодцев не пить. План такой — нам с тобой… , Здесь камни похожи на мыло, И сталь похожа на жесть, И слабость, как сила, И правда, как лесть. И не ясно, где мешок, а где шило, И не ясно, где обида, а где месть. И мне не нравится то, что здесь было, И мне не нравится то, что здесь есть. , Нам с тобой: голубых небес навес. Нам с тобой: станет лес глухой стеной. Нам с тобой: из оплеванных колодцев не пить. План такой — нам с тобой… Черная ночь да в реке вода — нам с тобой. И беда станет не беда. Уезжай!.. Так, была не была, прости и прощай!.. Пла

In [100]:
con.close()

In [97]:
res = cur.execute("Select * from text_songs")
result = res.fetchall()

In [98]:
len(result)

101

In [99]:
result

[('Виктор Цой',
  'vremya-est-a-deneg-net',
  'Дождь идет с утра, Будет, был и есть. И карман мой пуст, На часах шесть. Папирос нет, И огня нет, И в окне знакомом Не горит свет. , Время есть, а денег нет, И в гости некуда пойти. , И куда-то все Подевались вдруг. Я попал в какой-то Не такой круг. Я хочу пить, Я хочу есть, Я хочу просто Где-нибудь сесть. , Время есть, а денег нет, И в гости некуда пойти.',
  'https://rustih.ru/viktor-coj-vremya-est-a-deneg-net/'),
 ('Виктор Цой',
  'prosto-xochesh-ty-znat',
  'Идешь по улице один, Идешь к кому-то из друзей. Заходишь в гости без причин И просишь свежих новостей. Просто хочешь ты знать, Где и что происходит. , Звонишь по телефону всем, Кого-то — нет, а кто-то — здесь. Для разговоров много тем, Для разговоров время есть. Просто хочешь ты знать, Где и что происходит. , Узнал, что где-то пьют вино, А где-то музыка слышна. Тебя зовут туда, где пьют, И ты берешь еще вина. Просто хочешь ты знать, Где и что происходит. , Там кто-то спор ведет кру

In [8]:
con = sqlite3.connect('songs.db')

cur = con.cursor()
res = cur.execute('Select * from text_songs')
result = res.fetchall()

In [104]:
result[0][2]

'Дождь идет с утра, Будет, был и есть. И карман мой пуст, На часах шесть. Папирос нет, И огня нет, И в окне знакомом Не горит свет. , Время есть, а денег нет, И в гости некуда пойти. , И куда-то все Подевались вдруг. Я попал в какой-то Не такой круг. Я хочу пить, Я хочу есть, Я хочу просто Где-нибудь сесть. , Время есть, а денег нет, И в гости некуда пойти.'

In [9]:
text_songs = [i[2] for i in result]
text_songs[0]

'Дождь идет с утра, Будет, был и есть. И карман мой пуст, На часах шесть. Папирос нет, И огня нет, И в окне знакомом Не горит свет. , Время есть, а денег нет, И в гости некуда пойти. , И куда-то все Подевались вдруг. Я попал в какой-то Не такой круг. Я хочу пить, Я хочу есть, Я хочу просто Где-нибудь сесть. , Время есть, а денег нет, И в гости некуда пойти.'

In [108]:
autor_songs = [i[0] for i in result]
autor_songs[0]

'Виктор Цой'

In [109]:
name_songs = [i[1] for i in result]
name_songs[0]

'vremya-est-a-deneg-net'

In [12]:
text_prep_all = []
to_replace = ['!', '?', '.', ',', ':', '-', '  ', '[', ']', '))', '"', '(', ')', '—', '  ', 'ла', 'не']



for i in text_songs:
    text_prep = re.findall('[А-Я][^А-Я]*', i)
    text_prep = [a.lower() for a in text_prep]
    text_prep = [replace_all(a, to_replace) for a in text_prep]
    text_prep = [a.rstrip().rstrip() for a in text_prep]
    text_prep = [a.lstrip() for a in text_prep]
    clean_text = []
    for idx, val in enumerate(text_prep):
        if text_prep[idx] in [' ', '']:
            pass
        else:
            clean_text.append(val)
            
    text_prep = []
    for val in clean_text:
        if len(val) <= 10:
            pass
        else:
            text_prep.append(val)
    
    text_prep_unique = list(set(text_prep))
    text_prep_str = '\n'.join(text_prep_unique)
    text_prep_all.append(text_prep_str)
    

In [13]:
df_prepped = pd.DataFrame()
df_prepped['text'] = text_prep_all * 10
df_prepped['artist'] = 'Виктор Цой'
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('tsoy_text.csv', index=False)


print(df_prepped.shape)
df_prepped.head()

(1010, 2)


,artist,text
0,Виктор Цой,время есть а де г т\nя попал в какой то\nи ка...
1,Виктор Цой,кого то т а кто то здесь\nидешь к кому то и...
2,Виктор Цой,хочется в теплую ванну залезть\nя раздавлен зи...
3,Виктор Цой,пытается меня достать\nзлое белое колено\nкноп...
4,Виктор Цой,т дома никого т дома\nя снова человек без цел...


In [123]:
df_prepped

,artist,text
0,Виктор Цой,на часах шесть\nподевались вдруг\nне такой кру...
1,Виктор Цой,а где то музыка слышна\nи ты берешь еще вина\n...
2,Виктор Цой,бе я гадость лежит под окном\nможет быть это и...
3,Виктор Цой,здравствуйте мальчики\nв надежде тайну разгада...
4,Виктор Цой,а я лишний словно куча лома\nо мама мама я бе...
...,...,...
1005,Виктор Цой,и ты уже знаешь что это судьба поколений\nпопр...
1006,Виктор Цой,я встретил ее я встретил ее\nона там в кино\nя...
1007,Виктор Цой,из бумажных стаканов вода\nсолнце в кружке пив...
1008,Виктор Цой,каждый вечер хочется быть дома\nэти дни не так...
